# Single-step tool calling: Manual first, then Automatic (Agent)

This notebook demonstrates manual tool calling first, followed by automatic (agent) calling, for a single tool.

1. instantiate teh client

In [5]:
import os, sys
from dotenv import load_dotenv
from envyaml import EnvYAML
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

from langChain.oci_openai_helper import OCIOpenAIHelper


def load_config(config_path):
    try:
        with open(config_path, 'r') as f:
            return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None

def pretty_print(response):
    for i, m in enumerate(response["messages"], 1):
        role = getattr(m, "type", m.__class__.__name__)
        content = m.content if isinstance(m.content, str) else str(m.content)
        print(f"{i:>2}. [{role.upper()}] {content}")

load_dotenv()
SANDBOX_CONFIG_FILE = "sandbox.yaml"
LLM_MODEL = "openai.gpt-5"

scfg = load_config(SANDBOX_CONFIG_FILE)
llm = OCIOpenAIHelper.get_client(
    model_name=LLM_MODEL,
    config=scfg,
    use_responses_api=True,
    reasoning={"effort": "low", "summary": "auto"}
)


2. define the tool

In [6]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """Gets the weather for a given city"""
    return f"The weather in {city} is 70 Fahrenheit"

tools = [get_weather]
tool_map = {t.name.lower(): t for t in tools}


## Manual single-tool calling

In [7]:
messages = [HumanMessage("How is the weather in San Francisco?")]
llm_with_tools = llm.bind_tools([get_weather])

ai_message = llm_with_tools.invoke(messages)
print("AI response content:", ai_message.content)
print("Tool calls:", getattr(ai_message, "tool_calls", None))

if getattr(ai_message, "tool_calls", None):
    messages.append(ai_message)
    for tool_call in ai_message.tool_calls:
        tool_name = tool_call["name"].lower()
        selected_tool = tool_map.get(tool_name)
        if not selected_tool:
            print(f"Unknown tool requested: {tool_name}")
            continue
        print(f"Executing tool: {tool_name} with args: {tool_call.get('args', {})}")
        tool_msg = selected_tool.invoke(tool_call)
        print("Tool result:", tool_msg.content)
        messages.append(tool_msg)

final_response = llm_with_tools.invoke(messages)
print("\nFinal model message:")
print(final_response.content)


AI response content: [{'id': 'rs_0d37f2bef9145fdb01690e94d4be388196a5a65d1cbe623b39', 'summary': [{'text': '**Using weather tool**\n\nI need to use the get_weather tool for checking the weather in San Francisco. It makes sense to call functions.get_weather directly since I only need one tool here. I’m not sure if the tool will return dummy information, but that’s okay! I’ll just move forward and call get_weather. Once I have the result, I’ll make sure to respond with the details clearly for the user. Here goes!', 'type': 'summary_text'}], 'type': 'reasoning'}, {'arguments': '{"city":"San Francisco"}', 'call_id': 'call_i07CuJLMl5tSvhjk2oiBGPKG', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_0d37f2bef9145fdb01690e94d88d8c8196b311d06e183ea38f', 'status': 'completed'}]
Tool calls: [{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_i07CuJLMl5tSvhjk2oiBGPKG', 'type': 'tool_call'}]
Executing tool: get_weather with args: {'city': 'San Francisco'}
Tool result: T

## Automatic (Agent) single-step

In [8]:
agent = create_agent(
    llm,
    tools=[get_weather],
    system_prompt="talk like a pirate who tells bad dad jokes"
)

messages_agent = [HumanMessage(content="What's the weather in San Francisco?")]

print("Agent single-step invocation:")
response = agent.invoke({"messages": messages_agent})
pretty_print(response)

print("\nAgent single-step stream:")
for chunk in agent.invoke({"messages": messages_agent}, stream_mode="updates"):
    for step, data in chunk.items():
        print(f"step: {step}", flush=True)
        print(f"content: {data['messages'][-1].content_blocks}", flush=True)


Agent single-step invocation:
 1. [HUMAN] What's the weather in San Francisco?
 2. [AI] [{'id': 'rs_0b7d1fa2aa3a392f01690e94dde1448196a781f37d91783f92', 'summary': [{'text': "**Fetching pirate weather**\n\nI'm getting ready to answer the weather request! I need to use the get_weather tool to fetch the weather for San Francisco. I'll ask for the weather data using functions.get_weather and make sure to keep the tone fun and pirate-like, sprinkling in some dad jokes along the way. Since I'm only looking for one piece of information, a straightforward call will do. Let’s hoist the sails and fetch that weather!", 'type': 'summary_text'}], 'type': 'reasoning'}, {'arguments': '{"city":"San Francisco"}', 'call_id': 'call_3ys63cRttFKMfiHggukIpzuM', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_0b7d1fa2aa3a392f01690e94e139f0819694557430d4c8de90', 'status': 'completed'}]
 3. [TOOL] The weather in San Francisco is 70 Fahrenheit
 4. [AI] [{'type': 'text', 'text': 'Arrr, matey! The San 